PyTorch Version

In [1]:
from google.colab import drive
drive.mount('/content/drive')
file_path =  '/content/drive/MyDrive/T2-Code/Offensive-24K-T2.xlsx'
file_path =  '/content/drive/MyDrive/Dataset-v2/Offensive-24K-T1.xlsx'

Mounted at /content/drive


In [1]:
import warnings
warnings.filterwarnings('ignore')

import gc, os,time,numpy as np, pandas as pd, datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, MT5EncoderModel, logging

# ------------ GPU Setup ------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cuda':torch.cuda.empty_cache()

# ---------- Hyperparameters ---------
MAX_LEN = 128
EMBED_SIZE = 768
BERT_TRAINABLE = True
DRPT = 0.4
FC_ACT = 'elu'
LR_RATE = 6e-5
BATCH = 25  # reduced to lower GPU memory usage
NEPOCHS = 20
PATIENCE = 4
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

modelname = 'hfFineTuneMT5'
modelpath = os.path.join('.', 'Saved Models', modelname)
modelresults = os.path.join('.', 'Model Results')
modelsummaries = os.path.join('.', 'Model - Summaries-Figures')
for d in [modelpath, modelresults, modelsummaries]:
    os.makedirs(d, exist_ok=True)

# ------------ Utils ----------------
def hms_string(sec):
    h = int(sec // 3600)
    m = int((sec % 3600) // 60)
    s = sec % 60
    return f"{h} hrs {m:02d} mins {s:05.2f} secs"

logging.set_verbosity_error()


# -------- Dataset Class -----------
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# -------- Model Definition --------
class MT5Classifier(nn.Module):
    def __init__(self, model_name, dropout_rate=DRPT):
        super().__init__()
        self.bert = MT5EncoderModel.from_pretrained(model_name)
        if not BERT_TRAINABLE:
            for p in self.bert.parameters():
                p.requires_grad = False
        hidden = self.bert.config.hidden_size
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden, hidden)
        self.act = nn.ELU()
        self.out = nn.Linear(hidden, 1)
        self.sig = nn.Sigmoid()
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='linear')
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_uniform_(self.out.weight, nonlinearity='linear')
        nn.init.zeros_(self.out.bias)
    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pool = out.last_hidden_state[:, 0]
        x = self.dropout(pool)
        x = self.act(self.fc1(x))
        return self.sig(self.out(x)).squeeze(-1)

# --- Threshold Optimization -------
def optimize_threshold(y_true, y_probs):
    best, best_t = 0, .5
    for t in np.arange(0.1, 0.9, 0.001):
        f = f1_score(y_true, (y_probs >= t).astype(int))
        if f > best:
            best, best_t = f, t
    return best_t
# --------- Data Loading ------------
########################################################
def WriteResutls(reports):
  unt0 = {'precision':[], 'recall':[], 'f1-score':[] }
  tin1 = {'precision':[], 'recall':[], 'f1-score':[] }
  macroavg = {'precision':[], 'recall':[], 'f1-score':[] }
  weightedavg = {'precision':[], 'recall':[], 'f1-score':[] }
  accu = []
  for report in reports:
    for k,v in report.items():
      if 'UNT' in k:
        unt0['precision'].append(v['precision'])
        unt0['recall'].append(v['recall'])
        unt0['f1-score'].append(v['f1-score'])
      elif 'TIN' in k:
        tin1['precision'].append(v['precision'])
        tin1['recall'].append(v['recall'])
        tin1['f1-score'].append(v['f1-score'])
      elif 'macro avg' in k:
        macroavg['precision'].append(v['precision'])
        macroavg['recall'].append(v['recall'])
        macroavg['f1-score'].append(v['f1-score'])
      elif 'weighted avg' in k:
        weightedavg['precision'].append(v['precision'])
        weightedavg['recall'].append(v['recall'])
        weightedavg['f1-score'].append(v['f1-score'])
      else:
        accu.append(v)

  print('Accuracy:',np.mean(accu))
  print("")
  print('UNT 0 Precision:',np.mean(unt0['precision']))
  print('UNT 0 Recall:',np.mean(unt0['recall']))
  print('UNT 0 F1-Score:',np.mean(unt0['f1-score']))
  print("")
  print('TIN 1 Precision:',np.mean(tin1['precision']))
  print('TIN 1 Recall:',np.mean(tin1['recall']))
  print('TIN 1 F1-Score:',np.mean(tin1['f1-score']))

  print("")
  print('Weighted Avg Precision:',np.mean(weightedavg['precision']))
  print('Weighted Avg Recall:',np.mean(weightedavg['recall']))
  print('Weighted Avg F1-Score:',np.mean(weightedavg['f1-score']))

  print("")
  print('Macro  Precision:',np.mean(macroavg['precision']))
  print('Macro  Recall:',np.mean(macroavg['recall']))
  print('Macro  F1-Score:',np.mean(macroavg['f1-score']))


  file = open( result_path, mode='a' )
  file.write( modelname+ ' ( '+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +') \n' )
  file.write( 'Accuracy:'+str(np.mean(accu))+'\n' )
  file.write('UNT 0 Precision:'+str(np.mean(unt0['precision']))+'\n' )
  file.write('UNT 0 Recall:'+str(np.mean(unt0['recall']))+'\n' )
  file.write('UNT 0 F1-Score:'+str(np.mean(unt0['f1-score']))+'\n' )

  file.write('TIN 1 Precision:'+str(np.mean(tin1['precision']))+'\n' )
  file.write('TIN 1 Recall:'+str(np.mean(tin1['recall']))+'\n' )
  file.write('TIN 1 F1-Score:'+str(np.mean(tin1['f1-score']))+'\n' )

  file.write('Weighted Avg Precision:'+str(np.mean(weightedavg['precision']))+'\n' )
  file.write('Weighted Avg Recall:'+str(np.mean(weightedavg['recall']))+'\n' )
  file.write('Weighted Avg F1-Score:'+str(np.mean(weightedavg['f1-score']))+'\n' )

  file.write('Macro  Precision:'+str(np.mean(macroavg['precision']))+'\n' )
  file.write('Macro  Recall:'+str(np.mean(macroavg['recall']))+'\n' )
  file.write('Macro  F1-Score:'+str(np.mean(macroavg['f1-score']))+'\n' )
  file.close()
  print("Done")


######################################################################################
# -------------- Data Loading ---------------
result_path =  r'C:\Users\mojua\Desktop\DL-Code\T2-Classification-Result.csv'
file_path = r'C:\Users\mojua\Desktop\DL-Code\Dataset\Offensive-24K-T2.xlsx'

df = pd.read_excel(file_path, engine='openpyxl')
df['Tweet'] = df['Tweet'].astype(str)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.head())
print(df.info())
print(df.columns, df.shape)
gc.collect()

xcolumn = 'Tweet'
ycolumn = 'Tag'

# --------- Tokenizer Setup --------
bertmodelname = 'google/mt5-base'
tokenizer = AutoTokenizer.from_pretrained(bertmodelname)


Device: cuda
   Unnamed: 0                                              Tweet  Tag
0           0  USER دو بے نسلئیے ، حرامخور منافق مل رہے ہیں پ...    0
1           3                   USER گھٹیا انسان دنیا ہی چھوڑ دو    1
2          11  USER PMLN میں آپ کے بارے میں میری بہتر راۓ تھی...    1
3          15  USER اسپین کی ٹیم بھی پاکستان کی کرکٹ ٹیم کی ط...    0
4          20  ہمیں تو آج تک سمجھ نہیں آئی کہ کم عقل عیسائی ح...    1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8758 non-null   int64 
 1   Tweet       8758 non-null   object
 2   Tag         8758 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 205.4+ KB
None
Index(['Unnamed: 0', 'Tweet', 'Tag'], dtype='object') (8758, 3)


In [3]:
# --- K-Fold Training -------------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# metrics storage
valaccuracy, valprecision, valrecall, valf1 = [], [], [], []
testaccuracy, testprecision, testrecall, testf1 = [], [], [], []
val_f1_macro, test_f1_macro = [], []

start = time.time()
print('Start:', time.strftime("%I:%M %p"))
reports = []
for fold, (tr, te) in enumerate(skf.split(df[xcolumn], df[ycolumn]), 1):
    # split
    X_tr = df.loc[tr, xcolumn].tolist()
    y_tr = df.loc[tr, ycolumn].values
    X_te = df.loc[te, xcolumn].tolist()
    y_te = df.loc[te, ycolumn].values

    # train/val split
    X_tr, X_val, y_tr, y_val = train_test_split( X_tr, y_tr, test_size=0.15, random_state=0)

    # encodings
    enc_tr = tokenizer(X_tr, padding='max_length', truncation=True, max_length=MAX_LEN)
    enc_val = tokenizer( X_val, padding='max_length', truncation=True, max_length=MAX_LEN)
    enc_te = tokenizer(X_te, padding='max_length', truncation=True, max_length=MAX_LEN)

    # datasets
    ds_tr = TextDataset(enc_tr, y_tr)
    ds_val = TextDataset(enc_val, y_val)
    ds_te = TextDataset(enc_te, y_te)

    # Updated DataLoaders with pinned memory
    ld_tr = DataLoader(ds_tr, batch_size=BATCH, shuffle=True, num_workers=0, pin_memory=True)
    ld_val = DataLoader(ds_val, batch_size=BATCH, num_workers=0, pin_memory=True)
    ld_te = DataLoader(ds_te, batch_size=BATCH, num_workers=0, pin_memory=True)

    # model, loss, optimizer
    model = MT5Classifier(bertmodelname).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR_RATE)

    best_val, patience = 0, 0

    for epoch in range(NEPOCHS):
        model.train()
        for b in ld_tr:
            optimizer.zero_grad()
            ids = b['input_ids'].to(device, non_blocking=True)
            masks = b['attention_mask'].to(device, non_blocking=True)
            labels = b['labels'].to(device, non_blocking=True)

            probs = model(ids, masks)
            loss = criterion(probs, labels)
            loss.backward()
            optimizer.step()

            # immediate cleanup
            del ids, masks, labels, probs, loss
            torch.cuda.empty_cache()

        # validation
        model.eval()
        vp, vt = [], []
        with torch.no_grad():
            for b in ld_val:
                outs = model(
                    b['input_ids'].to(device),
                    b['attention_mask'].to(device)
                ).cpu().numpy()
                vp.extend(outs)
                vt.extend(b['labels'].numpy())

        thr = optimize_threshold(np.array(vt), np.array(vp))
        vpred = (np.array(vp) >= thr).astype(int)

        # compute validation metrics
        vm = f1_score(vt, vpred, average='macro')
       
        # save best
        if vm > best_val:
            best_val, patience = vm, 0
            save_path = os.path.join(modelpath, f"{modelname}_fold{fold}.pt")
            cpu_state = {k: v.cpu() for k, v in model.state_dict().items()}
            torch.save(cpu_state, save_path, _use_new_zipfile_serialization=False)
        else:
            patience += 1

        # lr decay
        if DECAY and patience % DECAY_AFTER == 0 and patience != 0:
            for g in optimizer.param_groups:
                g['lr'] *= DECAY_RATE

        print(f"Fold{fold} Ep{epoch+1}: Val MacroF1={vm:.4f} Pat={patience}")
        if patience >= PATIENCE:
            print(f"Stopping early at epoch {epoch+1}")
            break

    # load best model & test
    model.load_state_dict(
        torch.load(os.path.join(modelpath, f"{modelname}_fold{fold}.pt"))
    )
    model.eval()

    # evaluate on val & test
    vp, vt, tp, tt = [], [], [], []
    with torch.no_grad():
        for b in ld_val:
            vp.extend(model(
                b['input_ids'].to(device),
                b['attention_mask'].to(device)
            ).cpu().numpy())
            vt.extend(b['labels'].numpy())
        for b in ld_te:
            tp.extend(model(
                b['input_ids'].to(device),
                b['attention_mask'].to(device)
            ).cpu().numpy())
            tt.extend(b['labels'].numpy())

    thr = optimize_threshold(np.array(vt), np.array(vp))
    vpred = (np.array(vp) >= thr).astype(int)
    tpred = (np.array(tp) >= thr).astype(int)

    # compute test metrics
    ta = accuracy_score(tt, tpred)
    tp_ = precision_score(tt, tpred)
    tr = recall_score(tt, tpred)
    tf = f1_score(tt, tpred)
    tm = f1_score(tt, tpred, average='macro')
    tpc = precision_score(tt, tpred, average=None)
    trc = recall_score(tt, tpred, average=None)
    tfc = f1_score(tt, tpred, average=None)
    reports.append(classification_report( tt, tpred, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1'])) 
   
    print(f"Fold{fold} done at {time.strftime('%I:%M %p')}")

    # fold-level cleanup
    del model, optimizer, criterion, ld_tr, ld_val, ld_te, ds_tr, ds_val, ds_te
    torch.cuda.empty_cache()
    gc.collect()

print(f"Total runtime: {hms_string(time.time()-start)}")
WriteResutls(reports)

Start: 12:02 PM
Fold1 Ep1: Val MacroF1=0.4419 Pat=0
Fold1 Ep2: Val MacroF1=0.4377 Pat=1
Fold1 Ep3: Val MacroF1=0.4334 Pat=2
Fold1 Ep4: Val MacroF1=0.4377 Pat=3
Fold1 Ep5: Val MacroF1=0.4377 Pat=4
Stopping early at epoch 5
Fold1 done at 12:06 PM
Fold2 Ep1: Val MacroF1=0.4334 Pat=0
Fold2 Ep2: Val MacroF1=0.5017 Pat=0
Fold2 Ep3: Val MacroF1=0.5245 Pat=0
Fold2 Ep4: Val MacroF1=0.5936 Pat=0
Fold2 Ep5: Val MacroF1=0.5851 Pat=1
Fold2 Ep6: Val MacroF1=0.5903 Pat=2
Fold2 Ep7: Val MacroF1=0.5903 Pat=3
Fold2 Ep8: Val MacroF1=0.5903 Pat=4
Stopping early at epoch 8
Fold2 done at 12:13 PM
Fold3 Ep1: Val MacroF1=0.4384 Pat=0
Fold3 Ep2: Val MacroF1=0.5428 Pat=0
Fold3 Ep3: Val MacroF1=0.5607 Pat=0
Fold3 Ep4: Val MacroF1=0.5952 Pat=0
Fold3 Ep5: Val MacroF1=0.5973 Pat=0
Fold3 Ep6: Val MacroF1=0.5986 Pat=0
Fold3 Ep7: Val MacroF1=0.5938 Pat=1
Fold3 Ep8: Val MacroF1=0.6021 Pat=0
Fold3 Ep9: Val MacroF1=0.6707 Pat=0
Fold3 Ep10: Val MacroF1=0.6089 Pat=1
Fold3 Ep11: Val MacroF1=0.6072 Pat=2
Fold3 Ep12: Val Macr

In [1]:
import warnings
warnings.filterwarnings('ignore')
import gc, os,time, numpy as np,pandas as pd, datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModel, logging
# ------------- GPU Setup ------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cuda': torch.cuda.empty_cache()
# ---------- Hyperparameters ---------
MAX_LEN = 128
EMBED_SIZE = 768
BERT_TRAINABLE = True
DRPT = 0.4
FC_ACT = 'elu'
LR_RATE = 9e-6
BATCH = 32
NEPOCHS = 20
PATIENCE = 4
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

modelname = 'hfFineTuneMuril'
modelpath = os.path.join('.', 'Saved Models', modelname)
modelresults = os.path.join('.', 'Model Results')
modelsummaries = os.path.join('.', 'Model - Summaries-Figures')
for d in [modelpath, modelresults, modelsummaries]:
    os.makedirs(d, exist_ok=True)

# ---------- Utils ------------------
def hms_string(sec):
    h = int(sec // 3600)
    m = int((sec % 3600) // 60)
    s = sec % 60
    return f"{h} hrs {m:02d} mins {s:05.2f} secs"

logging.set_verbosity_error()

# -------- Dataset Class -----------
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# ------- Model Definition ----------
class MurilClassifier(nn.Module):
    def __init__(self, model_name, dropout_rate=DRPT):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        if not BERT_TRAINABLE:
            for p in self.bert.parameters(): p.requires_grad = False
        hidden = self.bert.config.hidden_size
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden, hidden)
        self.act = nn.ELU()
        self.out = nn.Linear(hidden, 1)
        self.sig = nn.Sigmoid()
        # he_uniform init
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='linear')
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_uniform_(self.out.weight, nonlinearity='linear')
        nn.init.zeros_(self.out.bias)
    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = out.pooler_output if out.pooler_output is not None else out.last_hidden_state[:,0]
        x = self.dropout(pooled)
        x = self.act(self.fc1(x))
        return self.sig(self.out(x)).squeeze(-1)

# ----- Threshold Optimization ------
def optimize_threshold(y_true, y_probs):
    best, best_thresh = 0, .5
    for t in np.arange(0.1, 0.9, 0.001):
        f = f1_score(y_true, (y_probs>=t).astype(int))
        if f>best: best, best_thresh = f, t
    return best_thresh

# ---------- Data Loading -----------
########################################################
def WriteResutls(reports):
  unt0 = {'precision':[], 'recall':[], 'f1-score':[] }
  tin1 = {'precision':[], 'recall':[], 'f1-score':[] }
  macroavg = {'precision':[], 'recall':[], 'f1-score':[] }
  weightedavg = {'precision':[], 'recall':[], 'f1-score':[] }
  accu = []
  for report in reports:
    for k,v in report.items():
      if 'UNT' in k:
        unt0['precision'].append(v['precision'])
        unt0['recall'].append(v['recall'])
        unt0['f1-score'].append(v['f1-score'])
      elif 'TIN' in k:
        tin1['precision'].append(v['precision'])
        tin1['recall'].append(v['recall'])
        tin1['f1-score'].append(v['f1-score'])
      elif 'macro avg' in k:
        macroavg['precision'].append(v['precision'])
        macroavg['recall'].append(v['recall'])
        macroavg['f1-score'].append(v['f1-score'])
      elif 'weighted avg' in k:
        weightedavg['precision'].append(v['precision'])
        weightedavg['recall'].append(v['recall'])
        weightedavg['f1-score'].append(v['f1-score'])
      else:
        accu.append(v)

  print('Accuracy:',np.mean(accu))
  print("")
  print('UNT 0 Precision:',np.mean(unt0['precision']))
  print('UNT 0 Recall:',np.mean(unt0['recall']))
  print('UNT 0 F1-Score:',np.mean(unt0['f1-score']))
  print("")
  print('TIN 1 Precision:',np.mean(tin1['precision']))
  print('TIN 1 Recall:',np.mean(tin1['recall']))
  print('TIN 1 F1-Score:',np.mean(tin1['f1-score']))

  print("")
  print('Weighted Avg Precision:',np.mean(weightedavg['precision']))
  print('Weighted Avg Recall:',np.mean(weightedavg['recall']))
  print('Weighted Avg F1-Score:',np.mean(weightedavg['f1-score']))

  print("")
  print('Macro  Precision:',np.mean(macroavg['precision']))
  print('Macro  Recall:',np.mean(macroavg['recall']))
  print('Macro  F1-Score:',np.mean(macroavg['f1-score']))


  file = open( result_path, mode='a' )
  file.write( modelname+ ' ( '+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +') \n' )
  file.write( 'Accuracy:'+str(np.mean(accu))+'\n' )
  file.write('UNT 0 Precision:'+str(np.mean(unt0['precision']))+'\n' )
  file.write('UNT 0 Recall:'+str(np.mean(unt0['recall']))+'\n' )
  file.write('UNT 0 F1-Score:'+str(np.mean(unt0['f1-score']))+'\n' )

  file.write('TIN 1 Precision:'+str(np.mean(tin1['precision']))+'\n' )
  file.write('TIN 1 Recall:'+str(np.mean(tin1['recall']))+'\n' )
  file.write('TIN 1 F1-Score:'+str(np.mean(tin1['f1-score']))+'\n' )

  file.write('Weighted Avg Precision:'+str(np.mean(weightedavg['precision']))+'\n' )
  file.write('Weighted Avg Recall:'+str(np.mean(weightedavg['recall']))+'\n' )
  file.write('Weighted Avg F1-Score:'+str(np.mean(weightedavg['f1-score']))+'\n' )

  file.write('Macro  Precision:'+str(np.mean(macroavg['precision']))+'\n' )
  file.write('Macro  Recall:'+str(np.mean(macroavg['recall']))+'\n' )
  file.write('Macro  F1-Score:'+str(np.mean(macroavg['f1-score']))+'\n' )
  file.close()
  print("Done")


######################################################################################
# -------------- Data Loading ---------------
result_path =  r'C:\Users\mojua\Desktop\DL-Code\T2-Classification-Result.csv'
file_path = r'C:\Users\mojua\Desktop\DL-Code\Dataset\Offensive-24K-T2.xlsx'

df = pd.read_excel(file_path, engine='openpyxl')
df['Tweet'] = df['Tweet'].astype(str)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.head())
print(df.info())
print(df.columns, df.shape)
gc.collect()

xcolumn = 'Tweet'
ycolumn = 'Tag'

Device: cuda
   Unnamed: 0                                              Tweet  Tag
0           0  USER دو بے نسلئیے ، حرامخور منافق مل رہے ہیں پ...    0
1           3                   USER گھٹیا انسان دنیا ہی چھوڑ دو    1
2          11  USER PMLN میں آپ کے بارے میں میری بہتر راۓ تھی...    1
3          15  USER اسپین کی ٹیم بھی پاکستان کی کرکٹ ٹیم کی ط...    0
4          20  ہمیں تو آج تک سمجھ نہیں آئی کہ کم عقل عیسائی ح...    1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8758 non-null   int64 
 1   Tweet       8758 non-null   object
 2   Tag         8758 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 205.4+ KB
None
Index(['Unnamed: 0', 'Tweet', 'Tag'], dtype='object') (8758, 3)


In [3]:
# ----- K-Fold Training -------------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# ---------- Tokenizer -------------
bertmodelname = 'google/muril-base-cased'
tokenizer = AutoTokenizer.from_pretrained(bertmodelname)
# metrics storage
val_acc, val_prec, val_rec, val_f1, val_macro = [], [], [], [], []
test_acc, test_prec, test_rec, test_f1, test_macro = [], [], [], [], []
reports = []

start = time.time()
print('Start:', time.strftime('%I:%M %p'))

for fold,(tr,te) in enumerate(skf.split(df[xcolumn], df[ycolumn]),1):
    X_tr, X_te = df.loc[tr,xcolumn].tolist(), df.loc[te,xcolumn].tolist()
    y_tr, y_te = df.loc[tr,ycolumn].values, df.loc[te,ycolumn].values
    X_tr, X_val, y_tr, y_val = train_test_split(X_tr,y_tr,test_size=0.15,random_state=0)
    # tokenize
    enc_tr = tokenizer(X_tr,padding='max_length',truncation=True,max_length=MAX_LEN)
    enc_val = tokenizer(X_val,padding='max_length',truncation=True,max_length=MAX_LEN)
    enc_te = tokenizer(X_te,padding='max_length',truncation=True,max_length=MAX_LEN)
    # datasets/loaders
    ds_tr = TextDataset(enc_tr,y_tr); 
    ds_val=TextDataset(enc_val,y_val); 
    ds_te=TextDataset(enc_te,y_te)
    ld_tr = DataLoader(ds_tr,batch_size=BATCH,shuffle=True);
    ld_val=DataLoader(ds_val,batch_size=BATCH); 
    ld_te=DataLoader(ds_te,batch_size=BATCH)
    # model, optim, loss
    model = MurilClassifier(bertmodelname).to(device)
    crit = nn.BCELoss(); 
    opt = torch.optim.Adam(model.parameters(),lr=LR_RATE)
    best_val,pat=0,0
    # train
    for ep in range(NEPOCHS):
        model.train()
        for b in ld_tr:
            opt.zero_grad()
            ids,mask,labels = b['input_ids'].to(device),b['attention_mask'].to(device),b['labels'].to(device)
            probs = model(ids,mask)
            loss = crit(probs,labels)
            loss.backward(); opt.step()
        # validate
        model.eval(); 
        vp,vt=[],[]
        with torch.no_grad():
            for b in ld_val:
                ids,mask = b['input_ids'].to(device),b['attention_mask'].to(device)
                vp.extend(model(ids,mask).cpu().numpy()); 
                vt.extend(b['labels'].numpy())
        thr=optimize_threshold(np.array(vt),np.array(vp))
        preds=(np.array(vp)>=thr).astype(int)
        # overall metrics
        va=accuracy_score(vt,preds); 
        vp_=precision_score(vt,preds)
        vr=recall_score(vt,preds); 
        vf=f1_score(vt,preds)
        vm=f1_score(vt,preds,average='macro')
        # per-class
        vpc=precision_score(vt,preds,average=None); 
        vrc=recall_score(vt,preds,average=None)
        vfc=f1_score(vt,preds,average=None)
        # update best
        if vm>best_val:
            best_val=vm; 
            pat=0
            torch.save(model.state_dict(),os.path.join(modelpath,f"{modelname}_fold{fold}.pt"))
        else:
            pat+=1
        if DECAY and pat%DECAY_AFTER==0 and pat!=0:
            for g in opt.param_groups: g['lr']*=DECAY_RATE
        print(f"Fold{fold} Ep{ep+1}: Val MacroF1={vm:.4f} Pat={pat}")
        if pat>=PATIENCE: break
    # load best
    model.load_state_dict(torch.load(os.path.join(modelpath,f"{modelname}_fold{fold}.pt")))
    model.eval(); 
    vp,vt, tp,tt=[],[],[],[]
    with torch.no_grad():
        for b in ld_val:
            out=model(b['input_ids'].to(device),b['attention_mask'].to(device))
            vp.extend(out.cpu().numpy());
            vt.extend(b['labels'].numpy())
        for b in ld_te:
            out=model(b['input_ids'].to(device),b['attention_mask'].to(device))
            tp.extend(out.cpu().numpy()); 
            tt.extend(b['labels'].numpy())
    # threshold and metrics
    thr=optimize_threshold(np.array(vt),np.array(vp))
    vpred=(np.array(vp)>=thr).astype(int); 
    tpred=(np.array(tp)>=thr).astype(int)
    reports.append(classification_report( tt, tpred, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))  
    print(f"Fold{fold} done at {time.strftime('%I:%M %p')}")
    # cleanup
    del model; torch.cuda.empty_cache(); gc.collect()
    
WriteResutls(reports)

Start: 05:21 PM
Fold1 Ep1: Val MacroF1=0.4763 Pat=0
Fold1 Ep2: Val MacroF1=0.5724 Pat=0
Fold1 Ep3: Val MacroF1=0.6186 Pat=0
Fold1 Ep4: Val MacroF1=0.7183 Pat=0
Fold1 Ep5: Val MacroF1=0.7283 Pat=0
Fold1 Ep6: Val MacroF1=0.6606 Pat=1
Fold1 Ep7: Val MacroF1=0.7242 Pat=2
Fold1 Ep8: Val MacroF1=0.7242 Pat=3
Fold1 Ep9: Val MacroF1=0.7222 Pat=4
Fold1 done at 05:27 PM
Fold2 Ep1: Val MacroF1=0.4299 Pat=0
Fold2 Ep2: Val MacroF1=0.5779 Pat=0
Fold2 Ep3: Val MacroF1=0.6347 Pat=0
Fold2 Ep4: Val MacroF1=0.7079 Pat=0
Fold2 Ep5: Val MacroF1=0.7546 Pat=0
Fold2 Ep6: Val MacroF1=0.7564 Pat=0
Fold2 Ep7: Val MacroF1=0.7622 Pat=0
Fold2 Ep8: Val MacroF1=0.7708 Pat=0
Fold2 Ep9: Val MacroF1=0.7692 Pat=1
Fold2 Ep10: Val MacroF1=0.7675 Pat=2
Fold2 Ep11: Val MacroF1=0.7691 Pat=3
Fold2 Ep12: Val MacroF1=0.7655 Pat=4
Fold2 done at 05:34 PM
Fold3 Ep1: Val MacroF1=0.4951 Pat=0
Fold3 Ep2: Val MacroF1=0.6631 Pat=0
Fold3 Ep3: Val MacroF1=0.7046 Pat=0
Fold3 Ep4: Val MacroF1=0.7212 Pat=0
Fold3 Ep5: Val MacroF1=0.7169 Pat=1

In [1]:
# -------------- Data Loading ---------------
import gc, os, time, numpy as np, pandas as pd, datetime
file_path = r'C:\Users\mojua\Desktop\DL-Code\Dataset\Offensive-24K-T2.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')
df['Tweet'] = df['Tweet'].astype(str)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.head())
print(df.info())
print(df.columns, df.shape)
gc.collect()

xcolumn = 'Tweet'
ycolumn = 'Tag'

   Unnamed: 0                                              Tweet  Tag
0           0  USER دو بے نسلئیے ، حرامخور منافق مل رہے ہیں پ...    0
1           3                   USER گھٹیا انسان دنیا ہی چھوڑ دو    1
2          11  USER PMLN میں آپ کے بارے میں میری بہتر راۓ تھی...    1
3          15  USER اسپین کی ٹیم بھی پاکستان کی کرکٹ ٹیم کی ط...    0
4          20  ہمیں تو آج تک سمجھ نہیں آئی کہ کم عقل عیسائی ح...    1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8758 non-null   int64 
 1   Tweet       8758 non-null   object
 2   Tag         8758 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 205.4+ KB
None
Index(['Unnamed: 0', 'Tweet', 'Tag'], dtype='object') (8758, 3)


In [5]:
import warnings
warnings.filterwarnings('ignore')

import gc, os, time, numpy as np, pandas as pd, datetime

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer, XLMRobertaConfig, XLMRobertaModel, logging

# ---------------- GPU Setup ----------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cuda': torch.cuda.empty_cache()

# ------------- Hyperparameters ------------
MAX_LEN = 128
EMBED_SIZE = 768  # same as hidden_size
BERT_TRAINABLE = True
DRPT = 0.4
FC_WEIGHTS_INIT = 'he_uniform'
FC_ACT = 'elu'
LR_RATE = 9e-6
BATCH = 32
NEPOCHS = 20
PATIENCE = 4
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1
result_path =  r'C:\Users\mojua\Desktop\DL-Code\T2-Classification-Result.csv'
modelname = 'hfFineTuneRoberta'
modelpath = os.path.join('.', 'Saved Models', modelname)
modelresults = os.path.join('.', 'Model Results')
modelsummaries = os.path.join('.', 'Model - Summaries-Figures')
for d in [modelpath, modelresults, modelsummaries]: os.makedirs(d, exist_ok=True)

# --------------- Utils ---------------------
def hms_string(sec_elapsed):
    h = int(sec_elapsed / 3600)
    m = int((sec_elapsed % 3600) / 60)
    s = sec_elapsed % 60
    return f"{h} hrs {m:02d} mins {s:05.2f} secs"

logging.set_verbosity_error()

# ----------- Dataset Class ----------------
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# --------- Model Definition ---------------
class RobertaClassifier(nn.Module):
    def __init__(self, model_name, dropout_rate=DRPT):
        super(RobertaClassifier, self).__init__()
        self.bert = XLMRobertaModel.from_pretrained(model_name)
        if not BERT_TRAINABLE:
            for param in self.bert.parameters():
                param.requires_grad = False
        hidden_size = self.bert.config.hidden_size
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.act = nn.ELU()
        self.out = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        # weight initialization matching 'he_uniform'
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='linear')
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_uniform_(self.out.weight, nonlinearity='linear')
        nn.init.zeros_(self.out.bias)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output if outputs.pooler_output is not None else outputs.last_hidden_state[:, 0]
        x = self.dropout(pooled)
        x = self.act(self.fc1(x))
        x = self.sigmoid(self.out(x).squeeze(-1))
        return x

# --------- Threshold Optimization ----------
def optimize_threshold(y_true, y_pred_probs):
    best_thresh = 0.5
    best_f1 = 0
    for thresh in np.arange(0.1, 0.9, 0.001):
        preds = (y_pred_probs >= thresh).astype(int)
        f1 = f1_score(y_true, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
    return best_thresh

# -------- K-Fold Training -----------------


Device: cuda


In [6]:

# ----------- Tokenizer Setup --------------
bertmodelname = 'xlm-roberta-base'
tokenizer = XLMRobertaTokenizer.from_pretrained(bertmodelname)

In [7]:
texts = df['Tweet'].tolist()

# Get token lengths
lengths = [len(tokenizer.encode(text, truncation=False)) for text in texts]

# Percentiles
p90 = int(np.percentile(lengths, 90))
p95 = int(np.percentile(lengths, 95))
p99 = int(np.percentile(lengths, 99))
max_len = p95  # or use p90 for stricter cutoff

print(f"90th percentile length: {p90}")
print(f"95th percentile length: {p95}")
print(f"95th percentile length: {p99}")


90th percentile length: 75
95th percentile length: 81
95th percentile length: 93


In [7]:
########################################################
def WriteResutls(reports):

  unt0 = {'precision':[], 'recall':[], 'f1-score':[] }
  tin1 = {'precision':[], 'recall':[], 'f1-score':[] }
  macroavg = {'precision':[], 'recall':[], 'f1-score':[] }
  weightedavg = {'precision':[], 'recall':[], 'f1-score':[] }
  accu = []
  for report in reports:
    for k,v in report.items():
      if 'UNT' in k:
        unt0['precision'].append(v['precision'])
        unt0['recall'].append(v['recall'])
        unt0['f1-score'].append(v['f1-score'])

      elif 'TIN' in k:
        tin1['precision'].append(v['precision'])
        tin1['recall'].append(v['recall'])
        tin1['f1-score'].append(v['f1-score'])

      elif 'macro avg' in k:
        macroavg['precision'].append(v['precision'])
        macroavg['recall'].append(v['recall'])
        macroavg['f1-score'].append(v['f1-score'])

      elif 'weighted avg' in k:
        weightedavg['precision'].append(v['precision'])
        weightedavg['recall'].append(v['recall'])
        weightedavg['f1-score'].append(v['f1-score'])
      else:
        accu.append(v)

  print('Accuracy:',np.mean(accu))
  print("")
  print('UNT 0 Precision:',np.mean(unt0['precision']))
  print('UNT 0 Recall:',np.mean(unt0['recall']))
  print('UNT 0 F1-Score:',np.mean(unt0['f1-score']))
  print("")
  print('TIN 1 Precision:',np.mean(tin1['precision']))
  print('TIN 1 Recall:',np.mean(tin1['recall']))
  print('TIN 1 F1-Score:',np.mean(tin1['f1-score']))

  print("")
  print('Weighted Avg Precision:',np.mean(weightedavg['precision']))
  print('Weighted Avg Recall:',np.mean(weightedavg['recall']))
  print('Weighted Avg F1-Score:',np.mean(weightedavg['f1-score']))

  print("")
  print('Macro  Precision:',np.mean(macroavg['precision']))
  print('Macro  Recall:',np.mean(macroavg['recall']))
  print('Macro  F1-Score:',np.mean(macroavg['f1-score']))


  file = open( result_path, mode='a' )
  file.write( modelname+ ' ( '+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +') \n' )
  file.write( 'Accuracy:'+str(np.mean(accu))+'\n' )
  file.write('UNT 0 Precision:'+str(np.mean(unt0['precision']))+'\n' )
  file.write('UNT 0 Recall:'+str(np.mean(unt0['recall']))+'\n' )
  file.write('UNT 0 F1-Score:'+str(np.mean(unt0['f1-score']))+'\n' )

  file.write('TIN 1 Precision:'+str(np.mean(tin1['precision']))+'\n' )
  file.write('TIN 1 Recall:'+str(np.mean(tin1['recall']))+'\n' )
  file.write('TIN 1 F1-Score:'+str(np.mean(tin1['f1-score']))+'\n' )

  file.write('Weighted Avg Precision:'+str(np.mean(weightedavg['precision']))+'\n' )
  file.write('Weighted Avg Recall:'+str(np.mean(weightedavg['recall']))+'\n' )
  file.write('Weighted Avg F1-Score:'+str(np.mean(weightedavg['f1-score']))+'\n' )

  file.write('Macro  Precision:'+str(np.mean(macroavg['precision']))+'\n' )
  file.write('Macro  Recall:'+str(np.mean(macroavg['recall']))+'\n' )
  file.write('Macro  F1-Score:'+str(np.mean(macroavg['f1-score']))+'\n' )
  file.close()
  print("Done")

######################################################################################

In [9]:
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

# Metric storage
valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
test_f1_macro = []
com_text, com_label, com_predicted, com_prob = [], [], [], []
com_indices = []
reports = []
start_time = time.time()
print('Local System Time:', time.strftime('%I:%M %p', time.localtime()))

for fold, (train_idx, test_idx) in enumerate(skf.split(df[xcolumn], df[ycolumn]), 1):
    x_train = df.loc[train_idx, xcolumn].tolist()
    y_train = df.loc[train_idx, ycolumn].values
    x_test = df.loc[test_idx, xcolumn].tolist()
    y_test = df.loc[test_idx, ycolumn].values
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=0)

    train_enc = tokenizer(x_train, padding='max_length', truncation=True, max_length=MAX_LEN)
    val_enc   = tokenizer(x_val, padding='max_length', truncation=True, max_length=MAX_LEN)
    test_enc  = tokenizer(x_test, padding='max_length', truncation=True, max_length=MAX_LEN)

    train_dataset = TextDataset(train_enc, y_train)
    val_dataset   = TextDataset(val_enc, y_val)
    test_dataset  = TextDataset(test_enc, y_test)

    train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=BATCH)
    test_loader  = DataLoader(test_dataset, batch_size=BATCH)

    model = RobertaClassifier(bertmodelname).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR_RATE)

    best_val_f1 = -np.inf
    patience_counter = 0

    for epoch in range(NEPOCHS):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            probs = model(input_ids, attention_mask)
            loss = criterion(probs, labels)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        val_probs, val_trues = [], []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].cpu().numpy()
                probs = model(input_ids, attention_mask).cpu().numpy()
                val_probs.extend(probs.tolist())
                val_trues.extend(labels.tolist())

        thresh = optimize_threshold(np.array(val_trues), np.array(val_probs))
        val_preds = (np.array(val_probs) >= thresh).astype(int)
        # overall metrics
        valaccuracy.append(accuracy_score(val_trues, val_preds))
        valprecision.append(precision_score(val_trues, val_preds))
        valrecall.append(recall_score(val_trues, val_preds))
        valf1.append(f1_score(val_trues, val_preds))
        valcm.append(confusion_matrix(val_trues, val_preds))

        val_f1_macro_score = f1_score(val_trues, val_preds, average='macro')


        # save best and early stop
        if val_f1_macro_score > best_val_f1:
            best_val_f1 = val_f1_macro_score
            patience_counter = 0
            torch.save(model.state_dict(), os.path.join(modelpath, f"{modelname}_fold{fold}.bin"))
        else:
            patience_counter += 1

        if DECAY and patience_counter % DECAY_AFTER == 0 and patience_counter != 0:
            for g in optimizer.param_groups:
                g['lr'] *= DECAY_RATE

        print(f"Fold {fold} Epoch {epoch+1}/{NEPOCHS} - Val Macro F1: {val_f1_macro_score:.4f} - Patience: {patience_counter}")
        if patience_counter >= PATIENCE:
            print(f"Stopping early at epoch {epoch+1}")
            break

    # Evaluate on test
    model.load_state_dict(torch.load(os.path.join(modelpath, f"{modelname}_fold{fold}.bin")))
    model.eval()
    test_probs, test_trues = [], []
    with torch.no_grad():
        for batch in test_loader:
            logits = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
            probs = logits.cpu().numpy()
            test_probs.extend(probs.tolist())
            test_trues.extend(batch['labels'].cpu().numpy().tolist())

    test_preds = (np.array(test_probs) >= thresh).astype(int)
    # overall test metrics
    testaccuracy.append(accuracy_score(test_trues, test_preds))
    testprecision.append(precision_score(test_trues, test_preds))
    testrecall.append(recall_score(test_trues, test_preds))
    testf1.append(f1_score(test_trues, test_preds))
    testcm.append(confusion_matrix(test_trues, test_preds))
    reports.append(classification_report( test_trues, test_preds, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))

    com_indices.extend(test_idx.tolist())
    com_text.extend(df.loc[test_idx, xcolumn].tolist())
    com_label.extend(df.loc[test_idx, ycolumn].tolist())
    com_predicted.extend(test_preds.tolist())
    com_prob.extend(test_probs)

    print(f"Completed fold {fold}/{skf.get_n_splits()}")
    # Cleanup
    del model
    torch.cuda.empty_cache()
    gc.collect()

print(f"Total runtime: {hms_string(time.time() - start_time)}")
WriteResutls(reports)

Local System Time: 08:12 PM
Fold 1 Epoch 1/20 - Val Macro F1: 0.4806 - Patience: 0
Fold 1 Epoch 2/20 - Val Macro F1: 0.6064 - Patience: 0
Fold 1 Epoch 3/20 - Val Macro F1: 0.6777 - Patience: 0
Fold 1 Epoch 4/20 - Val Macro F1: 0.6258 - Patience: 1
Fold 1 Epoch 5/20 - Val Macro F1: 0.7283 - Patience: 0
Fold 1 Epoch 6/20 - Val Macro F1: 0.7363 - Patience: 0
Fold 1 Epoch 7/20 - Val Macro F1: 0.7201 - Patience: 1
Fold 1 Epoch 8/20 - Val Macro F1: 0.7169 - Patience: 2
Fold 1 Epoch 9/20 - Val Macro F1: 0.7283 - Patience: 3
Fold 1 Epoch 10/20 - Val Macro F1: 0.7200 - Patience: 4
Stopping early at epoch 10
Completed fold 1/5
Fold 2 Epoch 1/20 - Val Macro F1: 0.5306 - Patience: 0
Fold 2 Epoch 2/20 - Val Macro F1: 0.6772 - Patience: 0
Fold 2 Epoch 3/20 - Val Macro F1: 0.7372 - Patience: 0
Fold 2 Epoch 4/20 - Val Macro F1: 0.7844 - Patience: 0
Fold 2 Epoch 5/20 - Val Macro F1: 0.8068 - Patience: 0
Fold 2 Epoch 6/20 - Val Macro F1: 0.7917 - Patience: 1
Fold 2 Epoch 7/20 - Val Macro F1: 0.7822 - Pa

In [9]:
import warnings
warnings.filterwarnings('ignore')

import gc, os,time, numpy as np,pandas as pd, datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
import torch,torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModel, logging

file_path = r'C:\Users\mojua\Desktop\DL-Code\Dataset\Offensive-24K-T2.xlsx'
result_path =  r'C:\Users\mojua\Desktop\DL-Code\T2-Classification-Result.csv'

# ---------------- GPU Setup ----------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cuda':torch.cuda.empty_cache()

# --------- Hyperparameters ---------
MAX_LEN = 130
BERT_TRAINABLE = True
DRPT = 0.4
FC_ACT = 'elu'
LR_RATE = 9e-6
BATCH = 32
NEPOCHS = 20
PATIENCE = 4
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1

modelname = 'hfFineTuneDistilBert'
modelpath = os.path.join('.', 'Saved Models', modelname)
for d in [modelpath, './Model Results', './Model - Summaries-Figures']:
    os.makedirs(d, exist_ok=True)
######################################################################################
# -------- Utils --------
def hms_string(sec):
    h = int(sec // 3600)
    m = int((sec % 3600) // 60)
    s = sec % 60
    return f"{h} hrs {m:02d} mins {s:05.2f} secs"

logging.set_verbosity_error()

# -------- Dataset --------
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings, self.labels = encodings, labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k,v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# -------- Model --------
class DistilBertClassifier(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name, config=config)
        if not BERT_TRAINABLE:
            for p in self.bert.parameters(): p.requires_grad=False
        hidden = self.bert.config.hidden_size
        self.dropout = nn.Dropout(DRPT)
        self.fc1 = nn.Linear(hidden, hidden)
        self.act = nn.ELU()
        self.out = nn.Linear(hidden, 1)
        self.sig = nn.Sigmoid()
        # he_uniform initialization
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='linear')
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_uniform_(self.out.weight, nonlinearity='linear')
        nn.init.zeros_(self.out.bias)
    def forward(self, input_ids, attention_mask):
        o = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = o.last_hidden_state[:,0]
        x = self.dropout(pooled)
        x = self.act(self.fc1(x))
        return self.sig(self.out(x).squeeze(-1))

# -------- Threshold --------
def optimize_threshold(y_true, y_probs):
    best_t, best_f = 0.5, 0
    for t in np.arange(0.1,0.9,0.001):
        p = (y_probs>=t).astype(int)
        f = f1_score(y_true,p)
        if f>best_f:
            best_f, best_t = f, t
    return best_t

Device: cuda


In [11]:
# -------- Data Loading --------
df = pd.read_excel(file_path, engine='openpyxl')
df['Tweet'] = df['Tweet'].astype(str)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.head())
print(df.info(), df.shape)
gc.collect()
xcol, ycol = 'Tweet', 'Tag'



   Unnamed: 0                                              Tweet  Tag
0           0  USER دو بے نسلئیے ، حرامخور منافق مل رہے ہیں پ...    0
1           3                   USER گھٹیا انسان دنیا ہی چھوڑ دو    1
2          11  USER PMLN میں آپ کے بارے میں میری بہتر راۓ تھی...    1
3          15  USER اسپین کی ٹیم بھی پاکستان کی کرکٹ ٹیم کی ط...    0
4          20  ہمیں تو آج تک سمجھ نہیں آئی کہ کم عقل عیسائی ح...    1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8758 non-null   int64 
 1   Tweet       8758 non-null   object
 2   Tag         8758 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 205.4+ KB
None (8758, 3)


In [13]:
########################################################
def WriteResutls(reports):

  unt0 = {'precision':[], 'recall':[], 'f1-score':[] }
  tin1 = {'precision':[], 'recall':[], 'f1-score':[] }
  macroavg = {'precision':[], 'recall':[], 'f1-score':[] }
  weightedavg = {'precision':[], 'recall':[], 'f1-score':[] }
  accu = []
  for report in reports:
    for k,v in report.items():
      if 'UNT' in k:
        unt0['precision'].append(v['precision'])
        unt0['recall'].append(v['recall'])
        unt0['f1-score'].append(v['f1-score'])

      elif 'TIN' in k:
        tin1['precision'].append(v['precision'])
        tin1['recall'].append(v['recall'])
        tin1['f1-score'].append(v['f1-score'])

      elif 'macro avg' in k:
        macroavg['precision'].append(v['precision'])
        macroavg['recall'].append(v['recall'])
        macroavg['f1-score'].append(v['f1-score'])

      elif 'weighted avg' in k:
        weightedavg['precision'].append(v['precision'])
        weightedavg['recall'].append(v['recall'])
        weightedavg['f1-score'].append(v['f1-score'])
      else:
        accu.append(v)

  print('Accuracy:',np.mean(accu))
  print("")
  print('UNT 0 Precision:',np.mean(unt0['precision']))
  print('UNT 0 Recall:',np.mean(unt0['recall']))
  print('UNT 0 F1-Score:',np.mean(unt0['f1-score']))
  print("")
  print('TIN 1 Precision:',np.mean(tin1['precision']))
  print('TIN 1 Recall:',np.mean(tin1['recall']))
  print('TIN 1 F1-Score:',np.mean(tin1['f1-score']))

  print("")
  print('Weighted Avg Precision:',np.mean(weightedavg['precision']))
  print('Weighted Avg Recall:',np.mean(weightedavg['recall']))
  print('Weighted Avg F1-Score:',np.mean(weightedavg['f1-score']))

  print("")
  print('Macro  Precision:',np.mean(macroavg['precision']))
  print('Macro  Recall:',np.mean(macroavg['recall']))
  print('Macro  F1-Score:',np.mean(macroavg['f1-score']))


  file = open( result_path, mode='a' )
  file.write( modelname+ ' ( '+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +') \n' )
  file.write( 'Accuracy:'+str(np.mean(accu))+'\n' )
  file.write('UNT 0 Precision:'+str(np.mean(unt0['precision']))+'\n' )
  file.write('UNT 0 Recall:'+str(np.mean(unt0['recall']))+'\n' )
  file.write('UNT 0 F1-Score:'+str(np.mean(unt0['f1-score']))+'\n' )

  file.write('TIN 1 Precision:'+str(np.mean(tin1['precision']))+'\n' )
  file.write('TIN 1 Recall:'+str(np.mean(tin1['recall']))+'\n' )
  file.write('TIN 1 F1-Score:'+str(np.mean(tin1['f1-score']))+'\n' )

  file.write('Weighted Avg Precision:'+str(np.mean(weightedavg['precision']))+'\n' )
  file.write('Weighted Avg Recall:'+str(np.mean(weightedavg['recall']))+'\n' )
  file.write('Weighted Avg F1-Score:'+str(np.mean(weightedavg['f1-score']))+'\n' )

  file.write('Macro  Precision:'+str(np.mean(macroavg['precision']))+'\n' )
  file.write('Macro  Recall:'+str(np.mean(macroavg['recall']))+'\n' )
  file.write('Macro  F1-Score:'+str(np.mean(macroavg['f1-score']))+'\n' )
  file.close()
  print("Done")

######################################################################################
# -------- Training Loop --------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# -------- Tokenizer & Config --------
bertmodelname = 'distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(bertmodelname)
config = AutoConfig.from_pretrained(bertmodelname)

In [15]:
# storage
valacc, valprec, valrec, valf1, valcm = [], [], [], [], []
val_prec_pc, val_rec_pc, val_f1_pc, val_f1_macro = [], [], [], []
testacc, testprec, testrec, testf1, testcm = [], [], [], [], []
test_prec_pc, test_rec_pc, test_f1_pc, test_f1_macro = [], [], [], []
reports =  []
start = time.time()
for fold, (tr, te) in enumerate(skf.split(df[xcol], df[ycol]), 1):
    Xtr = df.loc[tr, xcol].tolist()
    ytr = df.loc[tr, ycol].values
    Xte = df.loc[te, xcol].tolist()
    yte = df.loc[te, ycol].values

    Xtr, Xv, ytr, yv = train_test_split(Xtr, ytr, test_size=0.15, random_state=0)
    enc_tr = tokenizer(Xtr, padding='max_length', truncation=True, max_length=MAX_LEN)
    enc_v  = tokenizer(Xv,  padding='max_length', truncation=True, max_length=MAX_LEN)
    enc_te = tokenizer(Xte, padding='max_length', truncation=True, max_length=MAX_LEN)

    dt_tr = DataLoader(TextDataset(enc_tr, ytr), batch_size=BATCH, shuffle=True)
    dt_v  = DataLoader(TextDataset(enc_v,  yv), batch_size=BATCH)
    dt_te = DataLoader(TextDataset(enc_te, yte), batch_size=BATCH)

    model = DistilBertClassifier(bertmodelname).to(device)
    crit  = nn.BCELoss()
    opt   = torch.optim.Adam(model.parameters(), lr=LR_RATE)
    best_f, pat = -np.inf, 0

    for ep in range(NEPOCHS):
        model.train()
        for b in dt_tr:
            opt.zero_grad()
            ids   = b['input_ids'].to(device)
            mask  = b['attention_mask'].to(device)
            labels= b['labels'].to(device)
            probs = model(ids, mask)
            loss  = crit(probs, labels)
            loss.backward()
            opt.step()
        # validation
        model.eval()
        vp, vt = [], []
        with torch.no_grad():
            for b in dt_v:
                ids  = b['input_ids'].to(device)
                mask = b['attention_mask'].to(device)
                vp.extend(model(ids, mask).cpu().tolist())
                vt.extend(b['labels'].cpu().tolist())
        th    = optimize_threshold(np.array(vt), np.array(vp))
        vpred = (np.array(vp) >= th).astype(int)
        # record metrics
        valacc.append(       accuracy_score(vt, vpred))
        valprec.append(      precision_score(vt, vpred))
        valrec.append(       recall_score(vt, vpred))
        valf1.append(        f1_score(vt, vpred))
        valcm.append(        confusion_matrix(vt, vpred))
       
        vf1m = f1_score(vt, vpred, average='macro')
        val_f1_macro.append(vf1m)
        # early stopping
        if vf1m > best_f:
            best_f, pat = vf1m, 0
            torch.save(model.state_dict(),os.path.join(modelpath, f"{modelname}_fold{fold}.bin"))
        else:
            pat += 1
        if DECAY and pat % DECAY_AFTER == 0 and pat != 0:
            for g in opt.param_groups:
                g['lr'] *= DECAY_RATE
                
        print(f"Fold{fold} Ep{ep+1}/{NEPOCHS} - ValMacroF1={vf1m:.4f} Pat={pat}")
        if pat >= PATIENCE:
            break

    # test evaluation
    model.load_state_dict(torch.load(os.path.join(modelpath, f"{modelname}_fold{fold}.bin")))
    model.eval()
    tp, tt = [], []
    with torch.no_grad():
        for b in dt_te:
            ids  = b['input_ids'].to(device)
            mask = b['attention_mask'].to(device)
            tp.extend(model(ids, mask).cpu().tolist())
            tt.extend(b['labels'].cpu().tolist())
    tpred = (np.array(tp) >= th).astype(int)

    testacc.append(       accuracy_score(tt, tpred))
    testprec.append(      precision_score(tt, tpred))
    testrec.append(       recall_score(tt, tpred))
    testf1.append(        f1_score(tt, tpred))
    testcm.append(        confusion_matrix(tt, tpred))
    reports.append(classification_report( tt, tpred, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))

    print(f"Completed fold {fold}/5")
    del model
    torch.cuda.empty_cache()
    gc.collect()


print("Total runtime:", hms_string(time.time() - start))
WriteResutls(reports)

Fold1 Ep1/20 - ValMacroF1=0.5365 Pat=0
Fold1 Ep2/20 - ValMacroF1=0.6447 Pat=0
Fold1 Ep3/20 - ValMacroF1=0.6369 Pat=1
Fold1 Ep4/20 - ValMacroF1=0.6351 Pat=2
Fold1 Ep5/20 - ValMacroF1=0.6725 Pat=0
Fold1 Ep6/20 - ValMacroF1=0.6709 Pat=1
Fold1 Ep7/20 - ValMacroF1=0.6694 Pat=2
Fold1 Ep8/20 - ValMacroF1=0.6704 Pat=3
Fold1 Ep9/20 - ValMacroF1=0.6704 Pat=4
Completed fold 1/5
Fold2 Ep1/20 - ValMacroF1=0.5175 Pat=0
Fold2 Ep2/20 - ValMacroF1=0.6782 Pat=0
Fold2 Ep3/20 - ValMacroF1=0.6921 Pat=0
Fold2 Ep4/20 - ValMacroF1=0.7135 Pat=0
Fold2 Ep5/20 - ValMacroF1=0.7125 Pat=1
Fold2 Ep6/20 - ValMacroF1=0.7113 Pat=2
Fold2 Ep7/20 - ValMacroF1=0.7144 Pat=0
Fold2 Ep8/20 - ValMacroF1=0.7251 Pat=0
Fold2 Ep9/20 - ValMacroF1=0.7241 Pat=1
Fold2 Ep10/20 - ValMacroF1=0.7174 Pat=2
Fold2 Ep11/20 - ValMacroF1=0.7164 Pat=3
Fold2 Ep12/20 - ValMacroF1=0.7164 Pat=4
Completed fold 2/5
Fold3 Ep1/20 - ValMacroF1=0.5830 Pat=0
Fold3 Ep2/20 - ValMacroF1=0.7193 Pat=0
Fold3 Ep3/20 - ValMacroF1=0.7529 Pat=0
Fold3 Ep4/20 - ValMacro

In [ ]:
# start from here after kernel restart

In [1]:
import warnings
warnings.filterwarnings('ignore')

import gc, os,time, numpy as np,pandas as pd, datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split
import torch,torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModel, logging

file_path = r'C:\Users\mojua\Desktop\DL-Code\Dataset\Offensive-24K-T2.xlsx'
result_path =  r'C:\Users\mojua\Desktop\DL-Code\T2-Classification-Result.csv'

# ---------------- GPU Setup ----------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cuda':torch.cuda.empty_cache()

# --------- Hyperparameters ---------
MAX_LEN = 128
BERT_TRAINABLE = True
DRPT = 0.4
FC_ACT = 'elu'
LR_RATE = 1e-5
BATCH = 32
NEPOCHS = 20
PATIENCE = 4
DECAY = True
DECAY_RATE = 0.3
DECAY_AFTER = 1
modelname = 'hfFineTuneBert'
bertmodelname = 'bert-base-multilingual-cased'

modelpath = os.path.join('.', 'Saved Models', modelname)
for d in [modelpath, './Model Results', './Model - Summaries-Figures']:
    os.makedirs(d, exist_ok=True)
#################################################################################
# -------- Utils --------
def hms_string(sec):
    h = int(sec // 3600)
    m = int((sec % 3600) // 60)
    s = sec % 60
    return f"{h} hrs {m:02d} mins {s:05.2f} secs"

logging.set_verbosity_error()
# ----------- Dataset Class ----------------
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item
# -------- Model Definition ----------------
class BertClassifier(nn.Module):
    def __init__(self, model_name, dropout_rate=DRPT):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name, config=config)
        if not BERT_TRAINABLE:
            for p in self.bert.parameters(): p.requires_grad = False
        hidden = self.bert.config.hidden_size
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden, hidden)
        self.act = nn.ELU()
        self.out = nn.Linear(hidden, 1)
        self.sig = nn.Sigmoid()
        # he_uniform init
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='linear')
        nn.init.zeros_(self.fc1.bias)
        nn.init.kaiming_uniform_(self.out.weight, nonlinearity='linear')
        nn.init.zeros_(self.out.bias)
    def forward(self, input_ids, attention_mask):
        o = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = o.pooler_output if hasattr(o, 'pooler_output') else o.last_hidden_state[:,0]
        x = self.dropout(pooled)
        x = self.act(self.fc1(x))
        return self.sig(self.out(x).squeeze(-1))

# -------- Threshold Optimization -----------
def optimize_threshold(y_true, y_probs):
    best_t, best_f = 0.5, 0
    for t in np.arange(0.1,0.9,0.001):
        p = (y_probs>=t).astype(int)
        f = f1_score(y_true, p)
        if f>best_f: best_f, best_t = f, t
    return best_t


Device: cuda


In [3]:
# -------------- Data Loading ---------------
df = pd.read_excel(file_path, engine='openpyxl')
df['Tweet'] = df['Tweet'].astype(str)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.head())
print(df.info())
print(df.columns, df.shape)
gc.collect()
xcolumn = 'Tweet'
ycolumn = 'Tag'

   Unnamed: 0                                              Tweet  Tag
0           0  USER دو بے نسلئیے ، حرامخور منافق مل رہے ہیں پ...    0
1           3                   USER گھٹیا انسان دنیا ہی چھوڑ دو    1
2          11  USER PMLN میں آپ کے بارے میں میری بہتر راۓ تھی...    1
3          15  USER اسپین کی ٹیم بھی پاکستان کی کرکٹ ٹیم کی ط...    0
4          20  ہمیں تو آج تک سمجھ نہیں آئی کہ کم عقل عیسائی ح...    1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8758 non-null   int64 
 1   Tweet       8758 non-null   object
 2   Tag         8758 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 205.4+ KB
None
Index(['Unnamed: 0', 'Tweet', 'Tag'], dtype='object') (8758, 3)


In [5]:
########################################################
def WriteResutls(reports):

  unt0 = {'precision':[], 'recall':[], 'f1-score':[] }
  tin1 = {'precision':[], 'recall':[], 'f1-score':[] }
  macroavg = {'precision':[], 'recall':[], 'f1-score':[] }
  weightedavg = {'precision':[], 'recall':[], 'f1-score':[] }
  accu = []
  for report in reports:
    for k,v in report.items():
      if 'UNT' in k:
        unt0['precision'].append(v['precision'])
        unt0['recall'].append(v['recall'])
        unt0['f1-score'].append(v['f1-score'])

      elif 'TIN' in k:
        tin1['precision'].append(v['precision'])
        tin1['recall'].append(v['recall'])
        tin1['f1-score'].append(v['f1-score'])

      elif 'macro avg' in k:
        macroavg['precision'].append(v['precision'])
        macroavg['recall'].append(v['recall'])
        macroavg['f1-score'].append(v['f1-score'])

      elif 'weighted avg' in k:
        weightedavg['precision'].append(v['precision'])
        weightedavg['recall'].append(v['recall'])
        weightedavg['f1-score'].append(v['f1-score'])
      else:
        accu.append(v)

  print('Accuracy:',np.mean(accu))
  print("")
  print('UNT 0 Precision:',np.mean(unt0['precision']))
  print('UNT 0 Recall:',np.mean(unt0['recall']))
  print('UNT 0 F1-Score:',np.mean(unt0['f1-score']))
  print("")
  print('TIN 1 Precision:',np.mean(tin1['precision']))
  print('TIN 1 Recall:',np.mean(tin1['recall']))
  print('TIN 1 F1-Score:',np.mean(tin1['f1-score']))

  print("")
  print('Weighted Avg Precision:',np.mean(weightedavg['precision']))
  print('Weighted Avg Recall:',np.mean(weightedavg['recall']))
  print('Weighted Avg F1-Score:',np.mean(weightedavg['f1-score']))

  print("")
  print('Macro  Precision:',np.mean(macroavg['precision']))
  print('Macro  Recall:',np.mean(macroavg['recall']))
  print('Macro  F1-Score:',np.mean(macroavg['f1-score']))


  file = open( result_path, mode='a' )
  file.write( modelname+ ' ( '+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +') \n' )
  file.write( 'Accuracy:'+str(np.mean(accu))+'\n' )
  file.write('UNT 0 Precision:'+str(np.mean(unt0['precision']))+'\n' )
  file.write('UNT 0 Recall:'+str(np.mean(unt0['recall']))+'\n' )
  file.write('UNT 0 F1-Score:'+str(np.mean(unt0['f1-score']))+'\n' )

  file.write('TIN 1 Precision:'+str(np.mean(tin1['precision']))+'\n' )
  file.write('TIN 1 Recall:'+str(np.mean(tin1['recall']))+'\n' )
  file.write('TIN 1 F1-Score:'+str(np.mean(tin1['f1-score']))+'\n' )

  file.write('Weighted Avg Precision:'+str(np.mean(weightedavg['precision']))+'\n' )
  file.write('Weighted Avg Recall:'+str(np.mean(weightedavg['recall']))+'\n' )
  file.write('Weighted Avg F1-Score:'+str(np.mean(weightedavg['f1-score']))+'\n' )

  file.write('Macro  Precision:'+str(np.mean(macroavg['precision']))+'\n' )
  file.write('Macro  Recall:'+str(np.mean(macroavg['recall']))+'\n' )
  file.write('Macro  F1-Score:'+str(np.mean(macroavg['f1-score']))+'\n' )
  file.close()
  print("Done")

######################################################################################
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(bertmodelname)
config = AutoConfig.from_pretrained(bertmodelname)

In [7]:
# --------- K-Fold Training ----------------
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

# Metrics storage
valaccuracy, valprecision, valrecall, valf1, valcm = [], [], [], [], []
val_prec_pc, val_rec_pc, val_f1_pc, val_f1_macro = [], [], [], []
testaccuracy, testprecision, testrecall, testf1, testcm = [], [], [], [], []
test_prec_pc, test_rec_pc, test_f1_pc, test_f1_macro = [], [], [], []
reports = []
start = time.time()
print("Local System Time:", time.strftime("%I:%M %p", time.localtime()))

for fold, (tr, te) in enumerate(skf.split(df[xcolumn], df[ycolumn]),1):
    xtr = df.loc[tr, xcolumn].tolist();
    ytr = df.loc[tr, ycolumn].values
    xte = df.loc[te, xcolumn].tolist(); 
    yte = df.loc[te, ycolumn].values
    xtr, xv, ytr, yv = train_test_split(xtr, ytr, test_size=0.15, random_state=0)
    # tokenize
    enc_tr = tokenizer(xtr, padding='max_length', truncation=True, max_length=MAX_LEN)
    enc_v  = tokenizer(xv,  padding='max_length', truncation=True, max_length=MAX_LEN)
    enc_te = tokenizer(xte, padding='max_length', truncation=True, max_length=MAX_LEN)
    # datasets & loaders
    dt_tr = TextDataset(enc_tr, ytr); 
    lt = DataLoader(dt_tr, batch_size=BATCH, shuffle=True)
    dt_v  = TextDataset(enc_v,  yv); 
    lv = DataLoader(dt_v, batch_size=BATCH)
    dt_te = TextDataset(enc_te, yte); 
    le = DataLoader(dt_te, batch_size=BATCH)
    # model, loss, opt
    model = BertClassifier(bertmodelname).to(device)
    crit = nn.BCELoss()
    opt = torch.optim.Adam(model.parameters(), lr=LR_RATE)
    best_f, pt = -np.inf, 0
    # train
    for e in range(NEPOCHS):
        model.train()
        for b in lt:
            opt.zero_grad()
            ids = b['input_ids'].to(device); 
            m = b['attention_mask'].to(device)
            lbls = b['labels'].to(device)
            pr = model(ids,m)
            loss = crit(pr, lbls)
            loss.backward(); opt.step()
        # val
        model.eval(); vp, vt = [], []
        with torch.no_grad():
            for b in lv:
                ids = b['input_ids'].to(device); 
                m = b['attention_mask'].to(device)
                vp.extend(model(ids,m).cpu().numpy().tolist()); 
                vt.extend(b['labels'].cpu().numpy().tolist())
        th = optimize_threshold(np.array(vt), np.array(vp))
        vpred = (np.array(vp)>=th).astype(int)
        # record
        valaccuracy.append(accuracy_score(vt, vpred)); 
        valprecision.append(precision_score(vt, vpred))
        valrecall.append(recall_score(vt, vpred)); 
        valf1.append(f1_score(vt, vpred))
        valcm.append(confusion_matrix(vt, vpred))
        # per-class
       
        vm = f1_score(vt, vpred, average='macro'); 
        val_f1_macro.append(vm)
        # early stop & save
        if vm>best_f:
            best_f, pt = vm, 0
            torch.save(model.state_dict(), os.path.join(modelpath,f"{modelname}_fold{fold}.bin"))
        else:
            pt+=1
        if DECAY and pt%DECAY_AFTER==0 and pt!=0:
            for g in opt.param_groups: g['lr']*=DECAY_RATE
        print(f"Fold{fold} Ep{e+1}/{NEPOCHS} - ValMacroF1={vm:.4f} Pat={pt}")
        if pt>=PATIENCE:
            print(f"Stopping early at epoch {e+1}"); break
    # test eval
    model.load_state_dict(torch.load(os.path.join(modelpath,f"{modelname}_fold{fold}.bin")))
    model.eval(); tp, tt = [], []
    with torch.no_grad():
        for b in le:
            ids = b['input_ids'].to(device); 
            m = b['attention_mask'].to(device)
            tp.extend(model(ids,m).cpu().numpy().tolist()); 
            tt.extend(b['labels'].cpu().numpy().tolist())
    tpred = (np.array(tp)>=th).astype(int)
    testaccuracy.append(accuracy_score(tt,tpred)); 
    testprecision.append(precision_score(tt,tpred))
    testrecall.append(recall_score(tt,tpred)); 
    testf1.append(f1_score(tt,tpred))
    testcm.append(confusion_matrix(tt,tpred))
    reports.append(classification_report( tt, tpred, output_dict=True, zero_division=0, target_names=['UNT 0', 'TIN 1']))
    print(f"Completed fold {fold}/5")
    # cleanup
    del model; torch.cuda.empty_cache(); gc.collect()

print(f"Total runtime: {hms_string(time.time()-start)}")
WriteResutls(reports)

Local System Time: 09:28 PM
Fold1 Ep1/20 - ValMacroF1=0.5440 Pat=0
Fold1 Ep2/20 - ValMacroF1=0.6403 Pat=0
Fold1 Ep3/20 - ValMacroF1=0.7049 Pat=0
Fold1 Ep4/20 - ValMacroF1=0.6833 Pat=1
Fold1 Ep5/20 - ValMacroF1=0.7506 Pat=0
Fold1 Ep6/20 - ValMacroF1=0.6668 Pat=1
Fold1 Ep7/20 - ValMacroF1=0.6782 Pat=2
Fold1 Ep8/20 - ValMacroF1=0.7364 Pat=3
Fold1 Ep9/20 - ValMacroF1=0.7363 Pat=4
Stopping early at epoch 9
Completed fold 1/5
Fold2 Ep1/20 - ValMacroF1=0.5466 Pat=0
Fold2 Ep2/20 - ValMacroF1=0.6949 Pat=0
Fold2 Ep3/20 - ValMacroF1=0.7445 Pat=0
Fold2 Ep4/20 - ValMacroF1=0.7203 Pat=1
Fold2 Ep5/20 - ValMacroF1=0.6955 Pat=2
Fold2 Ep6/20 - ValMacroF1=0.7125 Pat=3
Fold2 Ep7/20 - ValMacroF1=0.7135 Pat=4
Stopping early at epoch 7
Completed fold 2/5
Fold3 Ep1/20 - ValMacroF1=0.6941 Pat=0
Fold3 Ep2/20 - ValMacroF1=0.7080 Pat=0
Fold3 Ep3/20 - ValMacroF1=0.7382 Pat=0
Fold3 Ep4/20 - ValMacroF1=0.7377 Pat=1
Fold3 Ep5/20 - ValMacroF1=0.7378 Pat=2
Fold3 Ep6/20 - ValMacroF1=0.7584 Pat=0
Fold3 Ep7/20 - ValMacroF